In [15]:
import os
import csv
import time
import html
import pandas as pd
from datasets import load_dataset
from google.cloud import translate_v2 as translate
from concurrent.futures import ThreadPoolExecutor    # Concurrent execution using threads
gcp_project_id = "artful-tractor-338209"   # Set the Google Cloud Project ID

## Create a Service Account:
After setting up project, enabling cloud translation api and setting billing account, In the Cloud Console, navigate to "IAM & Admin" > "Service accounts." Click "Create Service Account." Enter a name for service account, select the role(s) that service account needs (Cloud Translation API Editor), and click "Continue."

## Generate Key:
After creating the service account, click on it in the "Service accounts" page. Navigate to the "Keys" tab. Click on "Add Key" and choose "JSON." This will download a JSON key file containing the necessary credentials.

In [2]:
# give path to json key file

credential_path = "C:/Users/karti/Desktop/ComputerScience/internships/Nirant2/artful-tractor-338209-61f5638878ce.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

In [89]:
# provide dataset name among ai2_arc, gsm8k, lukaemon/mmlu
dataset_name = "ai2_arc"

In [90]:
if(dataset_name == "ai2_arc"):
    possible_configs = [
    'ARC-Challenge',
    'ARC-Easy'
    ]
    # columns to translate
    columns = ['question','choices']
    # columns not to translate, to keep in converted dataset as is.
    columns_asis = ['id','answerKey']

elif(dataset_name == "gsm8k"):
    possible_configs = [
    "main",
    "socratic"
    ]
    # columns to translate
    columns = ['question','answer']
    # columns not to translate, to keep in converted dataset as is.
    columns_asis = []
    
elif(dataset_name == "lukaemon/mmlu"):
    possible_configs = [
    "high_school_european_history",
    "business_ethics",
    "clinical_knowledge",
    "medical_genetics",
    "high_school_us_history",
    "high_school_physics",
    "high_school_world_history",
    "virology",
    "high_school_microeconomics",
    "econometrics",
    "college_computer_science",
    "high_school_biology",
    "abstract_algebra",
    "professional_accounting",
    "philosophy",
    "professional_medicine",
    "nutrition",
    "global_facts",
    "machine_learning",
    "security_studies",
    "public_relations",
    "professional_psychology",
    "prehistory",
    "anatomy",
    "human_sexuality",
    "college_medicine",
    "high_school_government_and_politics",
    "college_chemistry",
    "logical_fallacies",
    "high_school_geography",
    "elementary_mathematics",
    "human_aging",
    "college_mathematics",
    "high_school_psychology",
    "formal_logic",
    "high_school_statistics",
    "international_law",
    "high_school_mathematics",
    "high_school_computer_science",
    "conceptual_physics",
    "miscellaneous",
    "high_school_chemistry",
    "marketing",
    "professional_law",
    "management",
    "college_physics",
    "jurisprudence",
    "world_religions",
    "sociology",
    "us_foreign_policy",
    "high_school_macroeconomics",
    "computer_security",
    "moral_scenarios",
    "moral_disputes",
    "electrical_engineering",
    "astronomy",
    "college_biology",
    ]
    # columns to translate
    columns = ['input','A','B','C','D']
    # columns not to translate, to keep in converted dataset as is.
    columns_asis = ['target']

In [91]:
dataset = []
for config in possible_configs:
    dataset_slice = load_dataset(dataset_name, config,ignore_verifications=True)
    dataset.append(dataset_slice)

Found cached dataset parquet (C:/Users/karti/.cache/huggingface/datasets/parquet/ARC-Challenge-6372c4cac1109a78/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset parquet (C:/Users/karti/.cache/huggingface/datasets/parquet/ARC-Challenge-6372c4cac1109a78/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/3 [00:00<?, ?it/s]

In [92]:
# dataset
for g in dataset:
    print(g['train'])
    print(g['test'])
    # print(g['validation'])

Dataset({
    features: ['id', 'question', 'choices', 'answerKey'],
    num_rows: 3370
})
Dataset({
    features: ['id', 'question', 'choices', 'answerKey'],
    num_rows: 3548
})
Dataset({
    features: ['id', 'question', 'choices', 'answerKey'],
    num_rows: 3370
})
Dataset({
    features: ['id', 'question', 'choices', 'answerKey'],
    num_rows: 3548
})


In [9]:
def translate_text( text: str,target='hi') -> dict:
    """Translates text into the target language.
    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """

    translate_client = translate.Client()

    if isinstance(text, bytes):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)

    # html.unescape corrects translated symbols such as ', <,> etc.
    return html.unescape(result["translatedText"])
    
translate_text("How are you kartik")

'कार्तिक आप कैसे हैं?'

In [93]:
%%time

for i in range(len(possible_configs)):
    for set in dataset[i]:
        set_list = []
        
        for col in columns:
            values = [str(item[col]) for item in dataset[i][set]]
            
            # Use ThreadPoolExecutor for parallel translation
            if __name__ == '__main__':
                result =[]
                with ThreadPoolExecutor(max_workers=16) as exe:
                    # Maps the method with a list of values.
                    # result = list(exe.map(translate_text,values[:2]))
                    
                    batch_size = 1
                    for j in range(0,len(values[:3]),batch_size):
                        j_end = min(j + batch_size, len(values[:3]))
                        res = list(exe.map(translate_text,values[j:j_end]))
                        # time.sleep(2)
                        result.extend(res)
                    
            set_list.append(result)

        # Create folders for each configuration
        current_directory = os.getcwd()
        
        # Specify the path of the 'config' folder
        config_folder_path = os.path.join(current_directory, possible_configs[i])
        
        # Create the 'config' folder
        os.makedirs(config_folder_path, exist_ok=True)       
    
        # Transpose the 2D list
        transposed_data = list(map(list, zip(*set_list)))
        
        # to add untranslated columns in dataset
        for row in range(len(transposed_data)):
            for col in columns_asis:
                if col=='id':
                        position = 0
                else:
                    position = len(transposed_data[row])
                transposed_data[row].insert(position, dataset[i][set][col][row]) 
        
            
        path = os.path.join(possible_configs[i], f'{set}.csv')

        # append to previosly created csv file in case full dataset was not converted
        with open(path, 'a', encoding='utf-8') as f:
            # using csv.writer method from CSV package
            write = csv.writer(f)
            # write.writerow(columns)
            write.writerows(transposed_data)


CPU times: total: 2.05 s
Wall time: 26.2 s


In [94]:
df = pd.read_csv(path, header=None)

# Print the DataFrame
df.head()

,0,1,2,3
0,Mercury_7175875,एक खगोलशास्त्री ने देखा कि उल्कापिंड के प्रभाव...,"{'पाठ': ['ग्रहों का घनत्व कम हो जाएगा।', 'ग्रह...",C
1,Mercury_SC_409171,इंजीनियरों का एक समूह यह जानना चाहता था कि भूक...,"{'पाठ': ['इमारतें तेजी से बनाई जाएंगी', 'इमारत...",B
2,Mercury_SC_408547,प्रकाश संश्लेषण की प्रक्रिया का अंतिम परिणाम च...,{'पाठ': ['रासायनिक ऊर्जा जड़ों के माध्यम से अव...,C
